### Q2

### part a



Recall that we resolve the issue via ridge regression.
      \begin{equation*}
\min_\beta \lambda|\beta|^2_2 + \left|y - X\beta\right|^2_2.
\end{equation*}
The solution satisfies the first-order optimality requirements, according to our findings.$X^\top X\beta + \lambda\beta = X^\top y$. Rearranging, we have
\begin{equation*}
\lambda\beta = X^\top (y - X\beta),
\end{equation*}
We can modify the expression for $\alpha$ as
\begin{equation*}
\alpha = (1/\lambda)(y - X\beta) = (1/\lambda)(y - X(X^\top X + \lambda I)^{-1}X^\top y) = (X^\top X + \lambda I)^{-1}y,
\end{equation*}
The inference function is represented by $\langle\beta, x\rangle$ using $\alpha$, we have
\begin{equation*}
\langle\beta, x\rangle = x^\top\beta = x^\top X^\top\alpha = k(x)^\top(K + \lambda I)^{-1}y,
\end{equation*}
This formulation of the inference function is known as the kernel ridge regression, and it substitutes the explicit computation of the feature vectors with the kernel function evaluations.

In [81]:
# Imported libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import SVC
import sklearn.model_selection as ms
import warnings
warnings.filterwarnings('ignore')


### part b

In [82]:
# Read data
data = pd.read_csv("Data_Q2.csv")
print(data)

     Temperature  Humidity  Wind Speed     Flow  Consumption
0          5.578     93.00       0.082    0.185  5935.174070
1         15.510     64.38       0.085    0.133  6044.657863
2         15.730     64.21       0.084    0.152  6061.944778
3         15.620     65.22       0.083    0.145  6108.043217
4         15.450     67.69       0.083    0.189  6119.567827
..           ...       ...         ...      ...          ...
995       17.330     42.24       4.917   31.540  9443.855422
996        7.010     76.40       4.920   65.890  9449.638554
997       14.810     82.30       4.913    0.159  9449.638554
998       12.090     77.40       0.073    0.104  9449.638554
999       16.680     64.92       0.079  112.400  9449.990000

[1000 rows x 5 columns]


### part c

In [83]:
# standarization
stscaler = StandardScaler()
print(stscaler.fit(data))
print(stscaler.mean_)

StandardScaler()
[1.37793880e+01 7.53602200e+01 1.48328100e+00 2.57007980e+01
 8.18677638e+03]


In [84]:
data_standard = stscaler.fit_transform(data)
data_standard = pd.DataFrame(data_standard)
data_standard = pd.DataFrame(data_standard)
data_standard.head(10)

0         1         2         3         4
0 -1.931878  1.404846 -0.641791 -0.664370 -3.450208
1  0.407654 -0.874474 -0.640417 -0.665724 -3.282442
2  0.459476 -0.888013 -0.640875 -0.665229 -3.255953
3  0.433565 -0.807575 -0.641333 -0.665412 -3.185315
4  0.393521 -0.610863 -0.641333 -0.664266 -3.167655
5 -1.118742  0.799576 -0.641333 -0.665125 -3.008719
6 -1.104609  0.807540 -0.641333 -0.665047 -2.964570
7 -0.532210  0.592510 -0.640875 -0.664839 -2.823294
8  0.412365  0.170414 -0.641791 -0.664552 -2.805634
9 -1.872518  0.982750 -0.637211 -0.661844 -2.770315

### part d

In [85]:
# split into train and test data
T1, T2 = train_test_split(data_standard, test_size=0.2, random_state=42)

print('SD of Consumption column in T1:', T1[4].std())
print('SD of Consumption column in T2:', T2[4].std())

SD of Consumption column in T1: 1.0101813067055014
SD of Consumption column in T2: 0.961768317396748


### part e

In [86]:
# train kernelridge by using rbf kernel
kr_model = KernelRidge(kernel='rbf')
param_grid = {'gamma': np.logspace(0.1, 1, num=10)}
cv = KFold(n_splits = 5, shuffle= True)
grid_search = GridSearchCV(kr_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error',n_jobs=-1)
grid_search.fit(T1.drop(columns=[4]), T1[4])
print('Best gamma:', grid_search.best_params_['gamma'])
print('Best mean squared error:', -grid_search.best_score_)

Best gamma: 10.0
Best mean squared error: 0.7997814883725383


### part f

In [87]:
# print RMSE and R2 score on train and test data
Y_train_predict = grid_search.predict(T1.drop(columns=[4]))
Train_RMSE = np.sqrt(mean_squared_error(T1[4], Y_train_predict))
Train_R2 = r2_score(T1[4], Y_train_predict)
Y_test_predict = grid_search.predict(T2.drop(columns=[4]))
Test_RMSE = np.sqrt(mean_squared_error(T2[4], Y_test_predict))
Test_R2 = r2_score(T2[4], Y_test_predict)

In [88]:
print('RMSE on T1 set:', Train_RMSE)
print('R2 on T1 set:', Train_R2)
print('RMSE on T1 set:', Test_RMSE)
print('R2 on T1 set:', Test_R2)

RMSE on T1 set: 0.6500917245076449
R2 on T1 set: 0.5853383919265936
RMSE on T1 set: 0.8128918014697996
R2 on T1 set: 0.2820379758920609


### part g

In [89]:
# create labels for consumption
def map_consumption_to_class(consumption):
    if consumption <= 6500:
        return 1
    elif consumption > 6500 and consumption <= 7000:
        return 2
    elif consumption > 7000 and consumption <= 7500:
        return 3
    elif consumption > 7500 and consumption <= 8000:
        return 4
    elif consumption > 8000 and consumption <= 8500:
        return 5
    elif consumption > 8500 and consumption <= 9000:
        return 6
    else:
        return 7

In [90]:
frame_2 = pd.DataFrame(data)
frame_2['class'] = frame_2['Consumption'].apply(map_consumption_to_class)
frame_2

Temperature  Humidity  Wind Speed     Flow  Consumption  class
0          5.578     93.00       0.082    0.185  5935.174070      1
1         15.510     64.38       0.085    0.133  6044.657863      1
2         15.730     64.21       0.084    0.152  6061.944778      1
3         15.620     65.22       0.083    0.145  6108.043217      1
4         15.450     67.69       0.083    0.189  6119.567827      1
..           ...       ...         ...      ...          ...    ...
995       17.330     42.24       4.917   31.540  9443.855422      7
996        7.010     76.40       4.920   65.890  9449.638554      7
997       14.810     82.30       4.913    0.159  9449.638554      7
998       12.090     77.40       0.073    0.104  9449.638554      7
999       16.680     64.92       0.079  112.400  9449.990000      7

[1000 rows x 6 columns]

### part h

In [91]:
# standardized each data frame after spliting according to class
columns = frame_2.columns[:-1]
stscaler = StandardScaler()

In [92]:
for i in range(1, 8):
    k = (frame_2['class'] == i)
    frame_2.loc[k, columns] = stscaler.fit_transform(frame_2.loc[k, columns])
print(frame_2)

     Temperature  Humidity  Wind Speed      Flow  Consumption  class
0      -1.690575  1.563325   -0.508727 -0.365626    -1.925782      1
1       1.083295 -1.525195    0.581402 -0.412271    -1.304210      1
2       1.144738 -1.543541    0.218026 -0.395227    -1.206067      1
3       1.114017 -1.434547   -0.145350 -0.401507    -0.944353      1
4       1.066538 -1.167998   -0.145350 -0.362038    -0.878924      1
..           ...       ...         ...       ...          ...    ...
995     0.572096 -2.357184    1.585485 -0.112966     1.711956      7
996    -1.984162  0.416500    1.586861  0.552813     1.753338      7
997    -0.052107  0.895561    1.583650 -0.721200     1.753338      7
998    -0.725849  0.497697   -0.636763 -0.722266     1.753338      7
999     0.411091 -0.515640   -0.634011  1.454281     1.755853      7

[1000 rows x 6 columns]


### part i

In [93]:
# split data set such that it contains same sample
T3 = frame_2.loc[T1.index]
T4 = frame_2.loc[T2.index]
X_train = T3.drop(['class', 'Consumption'], axis=1)
Y_train = T3['class']
X_test = T4.drop(['class', 'Consumption'], axis=1)
Y_test = T4['class']

In [94]:
freq_train = np.zeros(7)
for k in Y_train:
  for i in range(1, 7+1):
    if k == i:
      freq_train[i-1] += 1

freq_train 

array([ 14.,  23.,  81., 170., 229., 203.,  80.])

### there is class imbalance

In [95]:
param_grid = {"gamma": np.logspace(0.1, 1, num=10)}
svc = SVC(kernel="rbf", class_weight="balanced")
grid_search = GridSearchCV(svc, param_grid, cv=5)
grid_search.fit(X_train, Y_train)
print("best gamma :", grid_search.best_params_['gamma'])
print("best mean squared error:", grid_search.best_score_)

best gamma : 10.0
best mean squared error: 0.48624999999999996


In [96]:
kr_svm = SVC(kernel='rbf',class_weight="balanced", probability=True, gamma = grid_search.best_params_['gamma'])
kr_svm.fit(X_train, Y_train)
Y_train_predict =kr_svm.predict(X_train)
Y_test_predict = kr_svm.predict(X_test)
Acc_train = accuracy_score(Y_train, Y_train_predict)
Acc_test = accuracy_score(Y_test, Y_test_predict)
print("Train Acc_score: ", Acc_train)
print("Test Acc_score: ", Acc_test)

Train Acc_score:  0.885
Test Acc_score:  0.575


### part j

In [97]:
# split each data frame according to class, train kernel ridge regression on each dataframe separately
models = []
for i in range(1,8):
    mask = Y_train == i
    X_train_i = X_train[mask]
    y_train_i = T3[mask]["Consumption"]
    param_grid = {"gamma": [0.1, 1, 10]}
    krr = KernelRidge(kernel="rbf")
    model = GridSearchCV(krr, param_grid, cv=5)
    model.fit(X_train_i, y_train_i)
    models.append(model)

### part k

In [98]:
# carry two-stage approach of classification followed by regression
def Y_predict_consumpation(x):  
  Y_predict = grid_search.predict(x)
  Y_predict_consumption = []
  for j,i in enumerate(Y_predict):
      model = models[i-1]
      y_pred_i = model.predict(x.iloc[j].values.reshape(1, -1))
      Y_predict_consumption.append(y_pred_i[0])
  return Y_predict_consumption

In [99]:
RMSE_T3 = mean_squared_error(T3["Consumption"],Y_predict_consumpation(X_train)) ** 0.5
RMSE_T4 = mean_squared_error(T4["Consumption"],Y_predict_consumpation(X_test)) ** 0.5

In [100]:
RMSE_T3 = mean_squared_error(T3["Consumption"],Y_predict_consumpation(X_train)) ** 0.5
RMSE_T4 = mean_squared_error(T4["Consumption"],Y_predict_consumpation(X_test)) ** 0.5

In [101]:
print('RMSE for T3 :',RMSE_T3)
print('RMSE for T4 :',RMSE_T4)

RMSE for T3 : 0.6720738380338367
RMSE for T4 : 1.0416413564433664


### part L

In [102]:
# comapre RMSE value
RMSE_S = mean_squared_error(T2[4], grid_search.predict(T2[[0, 1, 2, 3]])) ** 0.5
print("RMSE values:")
print("Using a simple regression method with the entire dataset:", RMSE_S)
print("Classification approach on T3 and T4:", RMSE_T3, RMSE_T4)

RMSE values:
Using a simple regression method with the entire dataset: 5.463998575002828
Classification approach on T3 and T4: 0.6720738380338367 1.0416413564433664
